In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no Instagram, LinkedIn e outras plataformas, pois eles costumam anunciar novos eventos e imersões por lá.
*   **Verificar a página de Imersões da Alura:** A Alura tem uma página dedicada às Imersões. Verifique se há alguma programação futura para a Imersão IA com Google Gemini.
*   **Assinar a newsletter da Alura:** Ao se inscrever na newsletter, você receberá informações sobre novos cursos, eventos e imersões diretamente no seu e-mail.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools":[{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A Imersão IA com Google Gemini mais recente da Alura ocorreu de 12 a 16 de maio de 2025. As inscrições para essa edição estiveram abertas até o dia 11 de maio de 2025.

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima imersão IA com Google Gemini da Alura']
Páginas utilizadas na resposta: starten.tech, youtube.com, alura.com.br



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk


In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca notícias no Google sobre o tópico indicado",
        tools=[google_search],
        instruction="""
        Voce é um assistente de pesquisa, sua tarefa é usar a ferramenta de busca do Google
        (google_search) para recuperar as ultimas noticias de lançamentos muito relevantes
        sobre o topico abaixo. Foque em no maximo 5 noticias, com base na quantidade e entusiasmo
        sobre ele.
        Se o tempo tiver poucas noticias ou reações entusiasmadas é possivel que não seja relevante.
        Por conta disso, ele pode ser subtituido por outro.
        Esses lançamentos relevantes devem ser atuais e de no maximo um mes da data de hoje.
        """
)
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Voce é um planejador de conteudo e especista em redes sociais.
        Com base na lista de lançamentos relevantes e relevantes buscados, voce deve:
        Usar a ferramenta de busca do google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderiamos abordar em um post sobre cada
        um deles.
        Tambem pode usar o google_search para buscar novas informações sobre os temas
        Ao final voce ira escolher o tema mais relevante dentre todos eles, baseado nas
        suas pesquisas.
        Retornar esse tema, seus pontos mais relevantes e o plano com os assuntos a serem
        abordados no post que sera escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho_de_post = call_agent(redator, entrada_do_agente_redator)
    return rascunho_de_post

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    post_final = call_agent(revisor, entrada_do_agente_revisor)
    return post_final

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")



🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: IA
Maravilha! Vamos então criar o post sobre novidades em IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Ok, vou procurar notícias recentes e relevantes sobre lançamentos de IA que tenham ocorrido no último mês.
> 
> 
> Com base nas notícias encontradas, aqui estão alguns dos lançamentos e avanços mais relevantes em IA no último mês (Abril/Maio de 2025):
> 
> 1.  **Avanços em modelos e ferramentas de IA:**
>     *   **Google Gemini:** A Google está expandindo a presença do Gemini em plataformas como Wear OS, Android Auto e Google TV. Também está aprofundando a integração da IA na sua ferramenta de pesquisa. [13]
>     *   **Google Gemma AI:** Os modelos de IA Gemma da Google ultrapassaram 150 milhões de downloads, com desenvolvedores criando mais de 70 mil variações na plataforma Hugging Face. [3]
>     *   **OpenAI Codex:** A OpenAI apresentou o Codex, um novo agente de codificação que promete revolucionar tarefas de engenharia de software ao gerar código mais limpo. [7]
>     *   **ChatGPT 4o e Gemini 2.5:** Maio de 2025 foi marcado por avanços significativos, com o ChatGPT atingindo 1 bilhão de buscas e o Gemini demonstrando capacidade de ler 1 milhão de linhas de código. [14]
> 2.  **Aplicações de IA em diversos setores:**
>     *   **Saúde:** Algoritmos de IA demonstram capacidade de superar vieses raciais no cuidado do câncer de próstata. [4]
>     *   **Educação:** Chatbots despertam emoções positivas e maior interesse por física e matemática. [4]
>     *   **Segurança:** Novo modelo "Scientist AI" promete guardrails para prevenir comportamentos autônomos indesejados. [4]
>     *   **Automação Inteligente:** A automação inteligente, impulsionada por IA, está revolucionando processos ao simplificar, automatizar e otimizar operações de maneira mais estratégica. [2]
> 3.  **Tendências e Previsões para 2025:**
>     *   **IA Explicável (XAI):** A IA Explicável busca tornar os processos de decisão da IA mais transparentes, permitindo que humanos compreendam como os sistemas chegaram a determinadas conclusões. [1]
>     *   **IA Generativa para Design e Criação:** Ferramentas de IA que geram textos, imagens e músicas estão revolucionando setores criativos, ampliando a capacidade humana de criação e reduzindo tempo e custo de produção. [1]
>     *   **Agentes de IA:** Agentes de IA ajudarão a simplificar a vida em casa e no trabalho. [11]
>     *   **Robôs Humanoides:** Robôs humanoides estão deixando de ser exclusividade de fábricas e entrando em setores como varejo, saúde e segurança. [15]
> 4.  **Eventos Relevantes:**
>     *   **Google I/O 2025:** O Google I/O, que ocorrerá em maio, deverá apresentar atualizações significativas para o Gemini e novas integrações de IA. [7, 13]
>     *   **Microsoft Build 2025:** Espera-se novas integrações de IA e melhorias no Copilot e na infraestrutura Azure. [3]
> 5.  **Relatório PNUD 2025:** O Relatório de Desenvolvimento Humano de 2025 investiga os impactos da digitalização e da inteligência artificial no desenvolvimento humano, abordando temas como saúde, educação, emprego e autonomia. [10]
> 6.  **Debates e Desafios:**
>     *   Há debates sobre o verdadeiro significado de "agente de IA". [3]
>     *   Especialistas analisam os impactos da GenAI no mercado de trabalho e na evolução econômica. [4]
>     *   O uso de obras protegidas para treinamento de IA levanta questões sobre direitos autorais. [8]
>     *   Há preocupações com a segurança e a ética da IA, incluindo a prevenção de comportamentos autônomos indesejados e a garantia de equidade e privacidade. [4, 15]
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Okay, com base nos lançamentos e avanços de IA listados para Abril/Maio de 2025, vou criar um plano de conteúdo para um post de mídia social, focando nos pontos mais relevantes e informativos para o público.
> 
> Primeiro, farei algumas pesquisas adicionais no Google para obter mais detalhes sobre alguns dos temas listados, como o Google Gemini, OpenAI Codex, e os debates sobre o significado de "agente de IA".
> 
> 
> Com base nas notícias e pesquisas, o tema mais relevante para um post de mídia social sobre IA em maio de 2025 é:
> 
> **Tema: A Expansão e Evolução dos Agentes de IA**
> 
> **Pontos Relevantes:**
> 
> *   **O que são Agentes de IA:** Sistemas autônomos que usam LLMs para tomar decisões e resolver tarefas complexas. Diferente de assistentes virtuais, eles operam com autonomia e interagem com o mundo real.
> *   **Aplicações em Diversos Setores:**  Marketing e vendas (otimização de campanhas), automação de tarefas complexas (análise de e-mails, respostas automatizadas), e até em setores como indústria e comércio.
> *   **Exemplos Práticos:** Google Gemini expandindo para Wear OS, Android Auto e Google TV, tornando-se um "agente" presente em diversos dispositivos. OpenAI Codex como um agente de codificação que auxilia desenvolvedores.
> *   **Debates e Desafios:** O verdadeiro significado de "agente de IA", o equilíbrio entre autonomia e supervisão, segurança e privacidade, e o impacto no mercado de trabalho (deskilling vs. novas oportunidades).
> *   **Tendências Futuras:** Agentes multi-modais (processando texto, imagens, vídeos e áudio), a importância do low-code para a escalabilidade, e a necessidade de treinamento e cultura de compartilhamento para a adoção bem-sucedida.
> 
> **Plano para o Post:**
> 
> 1.  **Introdução:**
>     *   Começar com uma pergunta chamativa: "Já imaginou ter um assistente digital que resolve problemas complexos por conta própria?"
>     *   Apresentar o conceito de Agentes de IA como a próxima evolução da inteligência artificial.
> 
> 2.  **O que são Agentes de IA:**
>     *   Explicar de forma simples o que são Agentes de IA e como eles se diferenciam dos assistentes virtuais tradicionais.
>     *   Mencionar a capacidade de operar com autonomia e interagir com o mundo real.
> 
> 3.  **Exemplos Concretos (Google Gemini e OpenAI Codex):**
>     *   Destacar a expansão do Google Gemini para diversos dispositivos (smartwatches, carros, TVs) como um exemplo de agente de IA presente no dia a dia.
>     *   Apresentar o OpenAI Codex como um agente de codificação que auxilia desenvolvedores, aumentando a eficiência e a qualidade do código.
> 
> 4.  **Aplicações e Impactos:**
>     *   Explorar as diversas aplicações dos Agentes de IA em setores como marketing, vendas, automação, indústria e comércio.
>     *   Discutir os impactos no mercado de trabalho, abordando o conceito de deskilling e as novas oportunidades que surgem com a necessidade de adaptação e novas habilidades.
> 
> 5.  **Debates e Desafios:**
>     *   Abordar os debates sobre o verdadeiro significado de "agente de IA" e o equilíbrio entre autonomia e supervisão.
>     *   Discutir as preocupações com segurança, privacidade e ética no desenvolvimento e implementação de Agentes de IA.
> 
> 6.  **Tendências e Futuro:**
>     *   Apresentar as tendências futuras, como agentes multi-modais, a importância do low-code e a necessidade de treinamento e cultura de compartilhamento.
>     *   Incentivar a discussão e o compartilhamento de opiniões sobre o futuro dos Agentes de IA.
> 
> 7.  **Chamada para Ação:**
>     *   Perguntar aos seguidores como eles imaginam o futuro com Agentes de IA e quais são suas maiores expectativas e preocupações.
>     *   Incentivar o compartilhamento do post e o debate sobre o tema.
> 
> Este plano permitirá criar um post informativo, relevante e engajador sobre a expansão e evolução dos Agentes de IA, um tema central no cenário tecnológico de Maio de 2025.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> Já imaginou ter um assistente digital que resolve problemas complexos por conta própria? 🤔
> 
> Prepare-se para a próxima evolução da inteligência artificial: os **Agentes de IA**! 🚀
> 
> 🤖 **O que são?**
> Sistemas autônomos que usam LLMs (modelos de linguagem grandes) para tomar decisões e resolver tarefas complexas. Eles operam com autonomia e interagem com o mundo real, diferente dos assistentes virtuais tradicionais.
> 
> Exemplos práticos? Temos! 👇
> ✨ **Google Gemini:** Expandindo para smartwatches, carros e TVs, tornando-se um agente presente no seu dia a dia.
> ✨ **OpenAI Codex:** Um agente de codificação que turbina o trabalho de desenvolvedores, aumentando a eficiência e a qualidade do código.
> 
> 💼 **Onde entram?**
> Marketing, vendas, automação, indústria, comércio… As aplicações são INÚMERAS! Mas e o mercado de trabalho? Calma! ➡️ Estamos falando de adaptação, novas habilidades e oportunidades.
> 
> 🗣️ **E os debates?**
> Qual o verdadeiro significado de "agente de IA"? Como equilibrar autonomia e supervisão? Segurança e privacidade são prioridades!
> 
> 🔮 **O futuro?**
> Agentes multi-modais (processando texto, imagens, vídeos e áudio), low-code para escalabilidade e a importância do treinamento e cultura de compartilhamento.
> 
> E aí, como você imagina o futuro com os Agentes de IA? Quais suas expectativas e preocupações? Compartilhe sua opinião nos comentários! 👇
> 
> #InteligenciaArtificial #Inovacao #Tecnologia #Alura
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!
> 


--------------------------------------------------------------
